## 1. Install Dependencies

In [ ]:
!pip install -q torch-geometric faiss-cpu scikit-learn matplotlib seaborn tqdm

## 2. Configuration

In [ ]:
# ============================================================================
# KAGGLE PATH CONFIGURATION - UPDATE THIS!
# ============================================================================
# Example: If you added dataset named 'graph-iot-dataset', use:
# DATA_DIR = "/kaggle/input/graph-iot-dataset/dataset-processed/flow_gnn"
DATA_DIR = "/kaggle/input/YOUR-DATASET-NAME/dataset-processed/flow_gnn"
OUTPUT_DIR = "/kaggle/working/output/flow_gnn"

# ============================================================================
# PROJECT SETTINGS
# ============================================================================
PROJECT_NAME = "Flow-GNN"
SEED = 42
DEVICE = "auto"  # auto, cuda, mps, cpu

# ============================================================================
# DATA SETTINGS
# ============================================================================
MAX_SAMPLES = 2_000_000

# ============================================================================
# MODEL SETTINGS
# ============================================================================
HIDDEN_DIM = 128
NUM_CLASSES = 2
NUM_LAYERS = 2
DROPOUT = 0.3

# ============================================================================
# GRAPH SETTINGS
# ============================================================================
K_NEIGHBORS = 10

# ============================================================================
# TRAINING SETTINGS
# ============================================================================
EPOCHS = 50
BATCH_SIZE = 512
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0001
PATIENCE = 10
MIN_DELTA = 0.001

## 3. Imports and Utilities

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch_geometric.data import Data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm import tqdm
import time
import json
import faiss
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_curve, auc, classification_report,
    precision_recall_curve, average_precision_score
)
from typing import Dict, Optional

print("✅ Imports successful")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
# Utility functions
def set_seed(seed: int = 42):
    """Set random seed for reproducibility."""
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def get_device(device_str: str = "auto") -> torch.device:
    """Get PyTorch device."""
    if device_str == "auto":
        if torch.cuda.is_available():
            device = torch.device("cuda")
            print(f"Using CUDA: {torch.cuda.get_device_name(0)}")
        elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
            device = torch.device("mps")
            print("Using Apple Metal (MPS)")
        else:
            device = torch.device("cpu")
            print("Using CPU")
    else:
        device = torch.device(device_str)
        print(f"Using device: {device}")
    return device

def compute_metrics(y_true: np.ndarray, y_pred: np.ndarray,
                    y_probs: Optional[np.ndarray] = None) -> Dict[str, float]:
    """Compute comprehensive metrics."""
    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, pos_label=1, zero_division=0),
        "recall": recall_score(y_true, y_pred, pos_label=1, zero_division=0),
        "f1": f1_score(y_true, y_pred, pos_label=1, zero_division=0)
    }
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    metrics["true_positive"] = int(tp)
    metrics["true_negative"] = int(tn)
    metrics["false_positive"] = int(fp)
    metrics["false_negative"] = int(fn)
    metrics["far"] = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    metrics["detection_rate"] = metrics["recall"]
    
    if y_probs is not None:
        fpr, tpr, _ = roc_curve(y_true, y_probs)
        metrics["auc"] = auc(fpr, tpr)
        metrics["average_precision"] = average_precision_score(y_true, y_probs)
    
    return metrics

class EarlyStopping:
    """Early stopping callback to prevent overfitting."""
    def __init__(self, patience: int = 10, min_delta: float = 1e-4):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
    
    def __call__(self, score: float) -> bool:
        if self.best_score is None:
            self.best_score = score
            return False
        if score > self.best_score + self.min_delta:
            self.best_score = score
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

print("✅ Utilities defined")

## 4. Build KNN Graph

In [ ]:
def build_knn_graph(X_scaled: np.ndarray, k: int = K_NEIGHBORS) -> torch.Tensor:
    """Build KNN graph with FAISS."""
    print(f"\n🔨 Building KNN graph (k={k})...")
    
    X = np.ascontiguousarray(X_scaled, dtype=np.float32)
    n_samples, n_features = X.shape
    print(f"   Data shape: {n_samples:,} samples × {n_features} features")
    
    with tqdm(total=3, desc="📍 Graph construction", ncols=100) as main_pbar:
        faiss.normalize_L2(X)
        main_pbar.update(1)
        main_pbar.set_postfix_str("Normalized vectors")
        
        index = faiss.IndexFlatIP(n_features)
        index.add(X)
        main_pbar.update(1)
        main_pbar.set_postfix_str("Built FAISS index")
        
        batch_size = 10000
        all_indices = []
        
        for i in tqdm(range(0, n_samples, batch_size),
                      desc="   🔍 KNN search", unit="batch", ncols=100, leave=False):
            end_idx = min(i + batch_size, n_samples)
            _, indices = index.search(X[i:end_idx], k + 1)
            all_indices.append(indices)
        
        indices = np.vstack(all_indices)
        main_pbar.update(1)
        main_pbar.set_postfix_str("KNN search complete")
    
    indices = indices[:, 1:]
    
    print("   🔗 Building edge list...")
    row = np.repeat(np.arange(n_samples), k)
    col = indices.flatten()
    
    edges = np.vstack([
        np.concatenate([row, col]),
        np.concatenate([col, row])
    ])
    
    edges = np.unique(edges, axis=1)
    edge_index = torch.tensor(edges, dtype=torch.long)
    
    num_edges = edge_index.shape[1]
    avg_degree = num_edges / n_samples
    
    print(f"✅ KNN graph built: {num_edges:,} edges, avg degree: {avg_degree:.2f}")
    return edge_index

print("✅ KNN graph function defined")

## 5. Define Model

In [ ]:
class FlowGraphSAGE(nn.Module):
    """GraphSAGE model for flow classification (node classification)."""
    
    def __init__(self, in_dim: int, hidden_dim: int = 128, num_classes: int = 2,
                 num_layers: int = 2, dropout: float = 0.3):
        super().__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        
        self.convs = nn.ModuleList()
        self.convs.append(SAGEConv(in_dim, hidden_dim))
        
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_dim, hidden_dim))
        
        if num_layers > 1:
            self.convs.append(SAGEConv(hidden_dim, hidden_dim))
        
        self.bns = nn.ModuleList()
        for _ in range(num_layers):
            self.bns.append(nn.BatchNorm1d(hidden_dim))
        
        self.classifier = nn.Linear(hidden_dim, 1)
        
        print(f"   FlowGraphSAGE: {in_dim}→{hidden_dim}x{num_layers}→1 (binary)")
    
    def forward(self, x: torch.Tensor, edge_index: torch.Tensor) -> torch.Tensor:
        """Forward pass. Returns logits (no sigmoid)."""
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        
        x = self.classifier(x)
        return x.squeeze(-1)

print("✅ Model defined")

## 6. Training Functions

In [ ]:
class RandomNodeSampler:
    """Simple random node sampler for mini-batch training."""
    def __init__(self, mask: torch.Tensor, batch_size: int, shuffle: bool = True):
        self.node_indices = mask.nonzero(as_tuple=True)[0]
        self.batch_size = batch_size
        self.shuffle = shuffle
    
    def __iter__(self):
        indices = self.node_indices.clone()
        if self.shuffle:
            perm = torch.randperm(len(indices))
            indices = indices[perm]
        for i in range(0, len(indices), self.batch_size):
            yield indices[i:i + self.batch_size]
    
    def __len__(self):
        return (len(self.node_indices) + self.batch_size - 1) // self.batch_size

def evaluate(model, data, mask, criterion, device, threshold=0.5):
    """Evaluate model on given mask."""
    model.eval()
    with torch.no_grad():
        logits = model(data.x, data.edge_index)
        mask_logits = logits[mask]
        mask_y = data.y[mask]
        loss = criterion(mask_logits, mask_y.float()).item()
    
    probs = torch.sigmoid(mask_logits).cpu().numpy()
    pred = (probs >= threshold).astype(int)
    true = mask_y.cpu().numpy()
    metrics = compute_metrics(true, pred, y_probs=probs)
    return loss, metrics

def evaluate_with_predictions(model, data, mask, criterion, threshold, device):
    """Evaluate and return predictions."""
    model.eval()
    with torch.no_grad():
        logits = model(data.x, data.edge_index)
        mask_logits = logits[mask]
        mask_y = data.y[mask]
        loss = criterion(mask_logits, mask_y.float()).item()
    
    probs = torch.sigmoid(mask_logits).cpu().numpy()
    pred = (probs >= threshold).astype(int)
    true = mask_y.cpu().numpy()
    metrics = compute_metrics(true, pred, y_probs=probs)
    return loss, metrics, true, pred, probs

def tune_threshold(model, data, mask, device):
    """Find optimal threshold to maximize F1 score."""
    model.eval()
    with torch.no_grad():
        logits = model(data.x, data.edge_index)
        val_logits = logits[mask]
        val_targets = data.y[mask]
    
    val_probs = torch.sigmoid(val_logits).cpu().numpy()
    y_val_np = val_targets.cpu().numpy()
    
    best_t, best_f1 = 0.5, 0.0
    thresholds = np.linspace(0.01, 0.99, 99)
    for t in tqdm(thresholds, desc="   Searching threshold", ncols=100, leave=False):
        y_pred = (val_probs >= t).astype(int)
        f1 = f1_score(y_val_np, y_pred, zero_division=0)
        if f1 > best_f1:
            best_f1 = f1
            best_t = t
    return best_t

print("✅ Training functions defined")

## 7. Load Data

In [ ]:
set_seed(SEED)
device = get_device(DEVICE)

print("=" * 70)
print("🔷 FLOW-BASED GNN PIPELINE")
print("=" * 70)

data_dir = Path(DATA_DIR)
print(f"\n📂 [1/4] Loading preprocessed data...")
print(f"   Data directory: {data_dir}")

X = np.load(data_dir / "X.npy")
y = np.load(data_dir / "y.npy")
idx_train = np.load(data_dir / "idx_train.npy")
idx_val = np.load(data_dir / "idx_val.npy")
idx_test = np.load(data_dir / "idx_test.npy")

x_tensor = torch.tensor(X, dtype=torch.float)
y_tensor = torch.tensor(y, dtype=torch.long)

print(f"   Features: {x_tensor.shape}")
print(f"   Train: {len(idx_train):,} | Val: {len(idx_val):,} | Test: {len(idx_test):,}")

## 8. Build Graph

In [ ]:
print("\n🔨 [2/4] Building KNN graph...")
edge_index = build_knn_graph(X, k=K_NEIGHBORS)

## 9. Create Masks

In [ ]:
print("\n🎯 [3/4] Creating train/val/test masks...")
train_mask = torch.zeros(len(y), dtype=torch.bool)
val_mask = torch.zeros(len(y), dtype=torch.bool)
test_mask = torch.zeros(len(y), dtype=torch.bool)
train_mask[idx_train] = True
val_mask[idx_val] = True
test_mask[idx_test] = True

## 10. Train Model

In [ ]:
print("\n🚀 [4/4] Training model...")
print("\n" + "=" * 70)
print("🚀 TRAINING FLOW-BASED GNN")
print("=" * 70)

# Create PyG Data
data = Data(
    x=x_tensor,
    edge_index=edge_index,
    y=y_tensor,
    train_mask=train_mask,
    val_mask=val_mask,
    test_mask=test_mask
).to(device)

# Calculate pos_weight
y_train = y_tensor[train_mask]
pos = (y_train == 1).sum().item()
neg = (y_train == 0).sum().item()
pos_weight = neg / pos if pos > 0 else 1.0

print(f"\n📊 Dataset Statistics:")
print(f"   Training samples:   {train_mask.sum().item():,}")
print(f"   Validation samples: {val_mask.sum().item():,}")
print(f"   Test samples:       {test_mask.sum().item():,}")
print(f"   Class distribution: Benign={neg:,} ({neg/(neg+pos)*100:.1f}%), Attack={pos:,} ({pos/(neg+pos)*100:.1f}%)")
print(f"   Positive weight:    {pos_weight:.4f}")

# Model
print(f"\n🏗️  Model Configuration:")
print(f"   Hidden dim:  {HIDDEN_DIM}")
print(f"   Num layers:  {NUM_LAYERS}")
print(f"   Dropout:     {DROPOUT}")

model = FlowGraphSAGE(
    in_dim=x_tensor.shape[1],
    hidden_dim=HIDDEN_DIM,
    num_classes=NUM_CLASSES,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT
).to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"   Total params: {total_params:,}")
print(f"   Device:       {device}")

# Optimizer & Loss
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY
)
criterion = nn.BCEWithLogitsLoss(
    pos_weight=torch.tensor([pos_weight], device=device)
)

print(f"\n⚙️  Training Configuration:")
print(f"   Epochs:         {EPOCHS}")
print(f"   Batch size:     {BATCH_SIZE}")
print(f"   Learning rate:  {LEARNING_RATE}")
print(f"   Weight decay:   {WEIGHT_DECAY}")
print(f"   Early stopping: {PATIENCE} epochs")

# Training setup
train_sampler = RandomNodeSampler(train_mask, batch_size=BATCH_SIZE, shuffle=True)
early_stopping = EarlyStopping(patience=PATIENCE, min_delta=MIN_DELTA)

best_f1 = 0.0
history = {
    'train_loss': [],
    'val_loss': [],
    'val_f1': [],
    'val_accuracy': []
}

print(f"\n🔥 Starting Training...")
print("-" * 70)

epoch_pbar = tqdm(range(1, EPOCHS + 1), desc="Training", unit="epoch", ncols=100)

for epoch in epoch_pbar:
    # Train
    model.train()
    total_loss = 0
    num_batches = 0
    
    for batch_nodes in train_sampler:
        batch_nodes = batch_nodes.to(device)
        optimizer.zero_grad()
        
        logits = model(data.x, data.edge_index)
        loss = criterion(logits[batch_nodes], data.y[batch_nodes].float())
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
    
    train_loss = total_loss / num_batches
    
    # Validate
    val_loss, val_metrics = evaluate(model, data, val_mask, criterion, device)
    
    # Update history
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_f1'].append(val_metrics['f1'])
    history['val_accuracy'].append(val_metrics['accuracy'])
    
    # Update progress bar
    epoch_pbar.set_postfix({
        'loss': f"{train_loss:.4f}",
        'val_f1': f"{val_metrics['f1']:.4f}",
        'val_acc': f"{val_metrics['accuracy']:.4f}"
    })
    
    # Save best model
    if val_metrics['f1'] > best_f1:
        best_f1 = val_metrics['f1']
        save_path = Path(OUTPUT_DIR) / 'best_model.pt'
        save_path.parent.mkdir(parents=True, exist_ok=True)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'metrics': val_metrics
        }, save_path)
    
    # Early stopping
    if early_stopping(val_metrics['f1']):
        print(f"\n⚠️  Early stopping at epoch {epoch}")
        break

print(f"\n✅ Training completed! Best validation F1: {best_f1:.4f}")

## 11. Tune Threshold

In [ ]:
print("\n" + "=" * 70)
print("🎯 TUNING DECISION THRESHOLD")
print("=" * 70)
best_threshold = tune_threshold(model, data, val_mask, device)
print(f"✅ Optimal threshold: {best_threshold:.4f}")

## 12. Final Evaluation

In [ ]:
print("\n" + "=" * 70)
print("🧪 FINAL EVALUATION ON TEST SET")
print("=" * 70)

start_time = time.time()
test_loss, test_metrics, y_true, y_pred, y_probs = evaluate_with_predictions(
    model, data, test_mask, criterion, best_threshold, device
)
inference_time = time.time() - start_time
latency_per_sample = inference_time / len(y_true)

print(f"\n📈 Test Results:")
print(f"   Accuracy:  {test_metrics['accuracy']:.4f}")
print(f"   Precision: {test_metrics['precision']:.4f}")
print(f"   Recall:    {test_metrics['recall']:.4f}")
print(f"   F1 Score:  {test_metrics['f1']:.4f}")
print(f"   AUC:       {test_metrics.get('auc', 0):.4f}")
print(f"   FAR:       {test_metrics['far']:.4f}")
print(f"\n⏱️  Inference Performance:")
print(f"   Total time:   {inference_time:.2f}s")
print(f"   Latency:      {latency_per_sample*1000:.4f} ms/sample")
print(f"   Throughput:   {len(y_true)/inference_time:.2f} samples/sec")

## 13. Save Results

In [ ]:
output_path = Path(OUTPUT_DIR)
output_path.mkdir(parents=True, exist_ok=True)

# Save metrics as JSON
test_metrics['latency_seconds'] = latency_per_sample
test_metrics['latency_ms'] = latency_per_sample * 1000
with open(output_path / 'metrics.json', 'w') as f:
    json.dump(test_metrics, f, indent=2)

# Save classification report
report = classification_report(y_true, y_pred, target_names=['Benign', 'Attack'], digits=4)
with open(output_path / 'classification_report.txt', 'w') as f:
    f.write("=" * 60 + "\n")
    f.write("   FLOW-GNN CLASSIFICATION REPORT\n")
    f.write("=" * 60 + "\n\n")
    f.write(report)

print(f"\n💾 Results saved to {output_path}/")
print("\n" + "=" * 70)
print("✨ ALL DONE!")
print("=" * 70 + "\n")

## 14. Visualizations

In [ ]:
# Confusion Matrix
fig, ax = plt.subplots(figsize=(8, 6))
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Benign', 'Attack'],
            yticklabels=['Benign', 'Attack'], ax=ax)
ax.set_title('Confusion Matrix - Flow GNN', fontsize=14, fontweight='bold')
ax.set_ylabel('True Label')
ax.set_xlabel('Predicted Label')
plt.tight_layout()
plt.savefig(output_path / 'confusion_matrix.png', dpi=300)
plt.show()

# ROC Curve
fig, ax = plt.subplots(figsize=(8, 6))
fpr, tpr, _ = roc_curve(y_true, y_probs)
roc_auc = auc(fpr, tpr)
ax.plot(fpr, tpr, color='#2196F3', lw=2.5, label=f'ROC Curve (AUC = {roc_auc:.4f})')
ax.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Random')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curve - Flow GNN', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(output_path / 'roc_curve.png', dpi=300)
plt.show()

# Training History
if len(history['train_loss']) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    epochs = range(1, len(history['train_loss']) + 1)
    
    axes[0].plot(epochs, history['train_loss'], 'b-', label='Train Loss', linewidth=2)
    axes[0].plot(epochs, history['val_loss'], 'r-', label='Val Loss', linewidth=2)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training & Validation Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(epochs, history['val_f1'], 'g-', label='Val F1', linewidth=2)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('F1 Score')
    axes[1].set_title('Validation F1 Score')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    axes[2].plot(epochs, history['val_accuracy'], 'm-', label='Val Accuracy', linewidth=2)
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('Accuracy')
    axes[2].set_title('Validation Accuracy')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(output_path / 'training_history.png', dpi=300)
    plt.show()

print("✅ Visualizations complete!")